In [22]:
import argparse
import dill
from datetime import datetime
import numpy as np
import pickle
import qiskit as qk
from qiskit.quantum_info import Operator, Statevector
from qiskit.circuit.quantumcircuit import QuantumCircuit
from typing import Tuple

from akash.TFIM_ham_gen import construct_hamiltonian

import dreamcoder as dc
from dreamcoder.frontier import Frontier, FrontierEntry
from dreamcoder.fragmentGrammar import FragmentGrammar
from dreamcoder.grammar import Grammar
from dreamcoder.program import Program
from dreamcoder.program import Abstraction
from dreamcoder.task import Task
from dreamcoder.utilities import numberOfCPUs
import dreamcoder.domains.quantum_ground_state.primitives as pr
from dreamcoder.domains.quantum_ground_state.primitives import (
    circuit_to_mat,
    full_op_names,
    mat_contraction,
    mat_to_tensor,
    execute_program,
    normalize_unitary,
    get_qiskit_circuit,
    get_instructions_from_qiskit,
    get_code_from_instructions,
    tcircuit,
    tensor_contraction,
    no_op,
    n_qubit_gate,
    QiskitTester,
)
from dreamcoder.domains.quantum_ground_state.primitives import execute_quantum_algorithm
from dreamcoder.domains.quantum_ground_state.tasks import GroundStateTask,get_energy
from dreamcoder.program import Program, Primitive
from dreamcoder.utilities import eprint, Curried

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
decomposed_list = [0, 1]


class args:
    n_qubits = 2
    J = 1
    hh = 0.05
    decomposed = 1
    arity = 2
    structurePenalty = 1
    pseudoCounts = 10


# Read the command line arguments
parser = argparse.ArgumentParser(
    description="Example implementation of Regularized Mutual Information Feature Selector on a solid drop.",
    epilog="Results will be saved in files with the OUTPUT tag in the 'outputs/' folder.",
    formatter_class=argparse.ArgumentDefaultsHelpFormatter,
)

parser.add_argument(
    "-n_qubits", type=int, default=args.n_qubits, help="Number of qubits"
)
parser.add_argument("-J", type=float, default=args.J, help="Interaction strength")
parser.add_argument("-hh", type=float, default=args.hh, help="External field strength")
parser.add_argument(
    "-decomposed",
    type=int,
    default=args.decomposed,
    help="Either 0=parametrized gates, 1=qiskit hardware basis",
)
parser.add_argument(
    "-arity",
    type=int,
    default=args.arity,
    help="Number of arguments of extracted gates",
)
parser.add_argument(
    "-structurePenalty", type=int, default=args.structurePenalty, help="hyperparameter"
)
parser.add_argument(
    "-pseudoCounts", type=int, default=args.pseudoCounts, help="hyperparameter"
)

try:
    args = parser.parse_args()
except SystemExit as e:
    eprint("Running from interactive session. Loading default parameters")

usage: ipykernel_launcher.py [-h] [-n_qubits N_QUBITS] [-J J] [-hh HH]
                             [-decomposed DECOMPOSED] [-arity ARITY]
                             [-structurePenalty STRUCTUREPENALTY]
                             [-pseudoCounts PSEUDOCOUNTS]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/leopoldo/.local/share/jupyter/runtime/kernel-v3d4ced2ea5e3000044480642e2c071ff5fa04bf91.json
762002226.py:48 > Running from interactive session. Loading default parameters


In [14]:
path = f"akash/solved_RL_circuits/circ_list_TFIM_qubit{args.n_qubits}_J{args.J}_h{args.hh}_decomposed{args.decomposed}.pickle"
name = f"ground_{args.n_qubits}_J{args.J}_h{args.hh}_dec{args.decomposed}"
with open(path, "rb") as handle:
    b = dill.load(handle)
eprint(f"Loading solutions from {path}")

4151542144.py:5 > Loading solutions from akash/solved_RL_circuits/circ_list_TFIM_qubit2_J1_h0.05_decomposed1.pickle


In [15]:
# Unfortunately these flags are set globally
dc.domains.quantum_ground_state.primitives.GLOBAL_NQUBIT_TASK = args.n_qubits
dc.domains.quantum_ground_state.primitives.GLOBAL_LIMITED_CONNECTIVITY = False

library_settings = {
    "topK": 2,  # how many solutions to consider
    "arity": args.arity,  # how many arguments
    "structurePenalty": args.structurePenalty,  # increase regularization 3 4 (it was 1), look at a few in [1,15]
    "pseudoCounts": args.pseudoCounts,  # increase to 100, test a few values
}

primitives = [pr.p_sx, pr.p_x, pr.p_rz, pr.p_cz]
grammar = Grammar.uniform(primitives)
eprint(f"Library building settings: {library_settings}")

2954477143.py:14 > Library building settings: {'topK': 2, 'arity': 2, 'structurePenalty': 1, 'pseudoCounts': 10}


In [16]:
# Generate a few example tasks
solutions = {}  # dict of task:solution
# NOTE: we have a task for each decomposition because they have various different real parameters
# We cannot have solutions with different requests for a task,
# and it is not clear how to use real numbers as primitives (just for evaluation, we cannot enumerate them)
for idx, circuit in enumerate(b):
    H = construct_hamiltonian(args.J, args.hh, args.n_qubits)
    instructions = get_instructions_from_qiskit(circuit)
    code, arguments = get_code_from_instructions(instructions)
    program = Program.parse(code)
    task = GroundStateTask(
        f"J_{args.J:2.2f}_h_{args.hh:2.2f}_N_{args.n_qubits}_v{idx}",
        hamiltonian=H,
        arguments=arguments,
        request=program.infer(),
    )
    likelihood = task.logLikelihood(program)
    prior = grammar.logLikelihood(program.infer(), program)

    frontier_entry = FrontierEntry(
        program=program, logLikelihood=likelihood, logPrior=prior
    )

    solutions[task] = Frontier(
        frontier=[frontier_entry],  # multiple solutions are allowed
        task=task,
    )
    eprint(f"#{idx:3}, Energy = {likelihood:2.6f}")
tasks = list(solutions.keys())

241499910.py:23 > #  0, Energy = 1.004986
241499910.py:23 > #  1, Energy = 1.004986
241499910.py:23 > #  2, Energy = 1.004985
241499910.py:23 > #  3, Energy = 1.004980
241499910.py:23 > #  4, Energy = 1.004979


241499910.py:23 > #  5, Energy = 1.004979
241499910.py:23 > #  6, Energy = 1.004979
241499910.py:23 > #  7, Energy = 1.004978
241499910.py:23 > #  8, Energy = 1.004934
241499910.py:23 > #  9, Energy = 1.004932
241499910.py:23 > # 10, Energy = 1.004931
241499910.py:23 > # 11, Energy = 1.004930
241499910.py:23 > # 12, Energy = 1.004929
241499910.py:23 > # 13, Energy = 1.004929
241499910.py:23 > # 14, Energy = 1.004929
241499910.py:23 > # 15, Energy = 1.004925
241499910.py:23 > # 16, Energy = 1.004898
241499910.py:23 > # 17, Energy = 1.004894
241499910.py:23 > # 18, Energy = 1.004872
241499910.py:23 > # 19, Energy = 1.004745
241499910.py:23 > # 20, Energy = 1.004743
241499910.py:23 > # 21, Energy = 1.004743
241499910.py:23 > # 22, Energy = 1.004743
241499910.py:23 > # 23, Energy = 1.004685
241499910.py:23 > # 24, Energy = 1.004554
241499910.py:23 > # 25, Energy = 1.004543
241499910.py:23 > # 26, Energy = 1.004542
241499910.py:23 > # 27, Energy = 1.004542


In [18]:
frontiers = [f for f in solutions.values()]

unique_frontiers_set = set()
unique_frontiers = []
for frontier in frontiers:
    program = frontier.entries[0].program
    if program not in unique_frontiers_set:
        unique_frontiers_set.add(program)
        unique_frontiers.append(frontier)
eprint(
    f"We have {len(unique_frontiers)}/{len(frontiers)} frontiers. The others are duplicate solutions"
)

unique_frontiers

2373419330.py:10 > We have 26/28 frontiers. The others are duplicate solutions


[Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (x (x (cz (sx (rz (sx (cz (x (sx (sx (cz (sx (cz (sx (cz (x (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $1) $1 $2) $1) $1 $2) $1) $1 $2) $1) $2) $2) $1 $2) $1) $3 $2) $2) $1 $2) $1) $2))))), logPrior=-53.42946076810878, logLikelihood=1.0049859541843977], task=J_1.00_h_0.05_N_2_v0),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (cz (cz (rz (cz (sx (cz (sx (rz (rz (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $5 $2) $4 $1) $1) $1 $2) $2) $1 $2) $3 $1) $1 $2) $1 $2))))))), logPrior=-39.91405480259485, logLikelihood=1.0049857375304754], task=J_1.00_h_0.05_N_2_v1),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (cz (rz (sx (cz (sx (rz (rz (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $5 $2) $4 $1) $1) $1 $2) $2) $3 $2) $1 $2))))))), logPrior=-33.92259025548687, logLikelihood=1.0049847779142502], task=J_1.00_h_0.05_N_2_v2),
 Frontier(entries=[Fronti

In [20]:
new_grammar, new_frontiers = FragmentGrammar.induceFromFrontiers(
    g0=grammar,
    frontiers=unique_frontiers[:4],
    **library_settings,
    CPUs=numberOfCPUs() - 2
)

new_grammar, new_frontiers

fragmentGrammar.py:298 > Inducing a grammar from 4 frontiers


fragmentGrammar.py:326 > Starting score -180.69170375518328
fragmentGrammar.py:333 > Proposed 8882 fragments.
fragmentGrammar.py:364 > New primitive of type tcircuit	(sx (rz (rz (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $3 $2) $4 $1) $1)	
(score = -147.030596; dScore = 33.661108; <uses> = 3.000000)
fragmentGrammar.py:377 > 	(<uses> in rewritten frontiers: 3.000000)
fragmentGrammar.py:333 > Proposed 213 fragments.
fragmentGrammar.py:364 > New primitive of type int -> tcircuit	(sx (cz $0 $1 $2))	
(score = -144.438984; dScore = 2.591612; <uses> = 6.211929)
fragmentGrammar.py:377 > 	(<uses> in rewritten frontiers: 7.000000)
fragmentGrammar.py:333 > Proposed 104 fragments.
fragmentGrammar.py:364 > New primitive of type tcircuit	(#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $0 $1 (#(lambda (lambda (lambda (lambda (lambda (sx (rz (rz (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $3 $2) $4 $1) $1)))))) $2 $3 $0 $1 $4) $0)	
(score = -143.137860; dScore = 1.301124; <uses> = 1.974913)
fragmentGrammar

(<dreamcoder.grammar.Grammar at 0x7f8a3211aa50>,
 [Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (x (x (cz (sx (rz (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (x (sx (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (x (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $1) $1) $1) $1) $2) $2) $1) $3 $2) $2) $1 $2) $1) $2))))), logPrior=-56.36601691320998, logLikelihood=1.0049859541843977], task=J_1.00_h_0.05_N_2_v0),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (cz (cz (rz (cz (#(lambda (lambda (lambda (lambda (lambda (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $0 $1 (#(lambda (lambda (lambda (lambda (lambda (sx (rz (rz (sx (x (sx (x (sx $0 $1) $2) $2) $1) $1) $3 $2) $4 $1) $1)))))) $2 $3 $0 $1 $4) $0)))))) $0 $5 $4 $1 $2) $1 $2) $3 $1) $1 $2) $1 $2))))))), logPrior=-21.887774067018395, logLikelihood=1

In [21]:
timestamp = datetime.now().isoformat()
with open(f"experimentOutputs/{timestamp}_{name}_grammar.pickle", "wb") as f:
    pickle.dump(new_grammar, f)

with open(f"experimentOutputs/{timestamp}_{name}_frontiers.pickle", "wb") as f:
    pickle.dump(new_frontiers, f)
eprint(f"Results saved in experimentOutputs/{timestamp}_{name}_...")

69069040.py:7 > Results saved in experimentOutputs/2024-09-17T19:03:39.955420_ground_2_J1_h0.05_dec1_...


---

## Some analysis

In [23]:
name = {}
simplification = {}
depth = {}

# The following are auxiliary functions to handle invented primitives
def get_program_children(program):
    try:
        return {k: getName(k) for _, k in program.body.walk() if k.isInvented}
    except:
        return {k: getName(k) for _, k in program.walk() if k.isInvented}


def get_program_simplification(program):
    if program in name:
        return name[program]
    children = get_program_children(program)
    simplification_ = program
    for k, childName in children.items():
        simplification_ = simplification_.substitute(
            k, Primitive(childName, ty=None, value=None)
        )
    return simplification_

def getName(program):
    if program in name:
        return name[program]
    children = get_program_children(program)
    simplification_ = get_program_simplification(program.body)
    # for original, simplified in nameSimplification.items():
    #     simplification_ = simplification_.substitute(Primitive(original,None,None),
    #                                                     Primitive(simplified,None,None))
    name[program] = "f%d" % len(name)
    # name[program] = f"f{len(name):03d}" # need to make name unambigous (f1 and f13 is not nice for easy string replacement)

    simplification[program] = simplification_
    depth[program] = 1 + max([depth[k] for k in children] + [0])
    return name[program]


def get_primitive_arguments(tp, arguments, last_qubit=0):
    # Get argument to run a primitive
    if len(tp.arguments) != 0:
        if tp.arguments[0].name == "int":
            arguments.append(last_qubit)
            last_qubit += 1
        elif tp.arguments[0].name =="real":
            arguments.append(42) # TODO: change
        elif tp.arguments[0].name == "tcircuit":
            arguments.append(["no_op"])
        return get_primitive_arguments(tp.arguments[1], arguments, last_qubit)

    else: # TODO: why is this function getting called in two different ways?
        for i in range(len(arguments)):
            if type(arguments[i]) == list:
                arguments[i] = no_op(last_qubit)
        return arguments
    
    

def get_primitive_lambda(n_qubit, operation_name):
    if n_qubit == 1:
        return lambda old_circuit, q: n_qubit_gate(
            old_circuit, q, operation_name=operation_name
        )
    elif n_qubit == 2:
        return lambda old_circuit, q1, q2: n_qubit_gate(
            old_circuit, q1, q2, operation_name=operation_name
        )
    elif n_qubit == 3:
        return lambda old_circuit, q1, q2, q3: n_qubit_gate(
            old_circuit, q1, q2, q3, operation_name=operation_name
        )
    elif n_qubit == 4:
        return lambda old_circuit, q1, q2, q3, q4: n_qubit_gate(
            old_circuit, q1, q2, q3, q4, operation_name=operation_name
        )
    elif n_qubit == 5:
        return lambda old_circuit, q1, q2, q3, q4, q5: n_qubit_gate(
            old_circuit, q1, q2, q3, q4, q5, operation_name=operation_name
        )
    elif n_qubit == 6:
        return lambda old_circuit, q1, q2, q3, q4, q5, q6: n_qubit_gate(
            old_circuit, q1, q2, q3, q4, q5, q6, operation_name=operation_name
        )
    elif n_qubit == 7:
        raise Exception("ask Kevin.")


def f_circuit(circuit_unitary, *qubit, circuit_template):
    # eprint(circuit_template)
    n_args, circuit_ops = circuit_template
    circuit_ops = list(circuit_ops)
    # apply template
    # returns a tuple of circuit ops ( ("gate", 1, 2, ...), ...   )
    circuit_ops = (((op[0], *[qubit[q] for q in op[1:]])) for op in circuit_ops)

    for op in circuit_ops:
        circuit_unitary = full_op_names[op[0]](circuit_unitary, *op[1:])
    return circuit_unitary

In [24]:
path = "experimentOutputs/2024-09-17T19:03:39.955420_ground_2_J1_h0.05_dec1_"
with open(f"{path}grammar.pickle", "rb") as f:
    new_grammar = pickle.load(f)
    
with open(f"{path}frontiers.pickle", "rb") as f:
    new_frontiers = pickle.load(f)

In [34]:
invented_primitives = [p for p in new_grammar.primitives if p.isInvented]
for p in invented_primitives:
    getName(p)
depths = {depth[p] for p in invented_primitives}
depth2primitives = {
    d: {p for p in invented_primitives if depth[p] == d} for d in depths
}

# Define primitive circuit operations as globals
for i, (original, simplified) in enumerate(list(simplification.items())[:]):
    arguments = get_primitive_arguments(original.tp, [])
    n_arguments = len(arguments) - 1
    
    Primitive.GLOBALS[name[original]] = Primitive(
        name=name[original],
        ty=original.tp,
        value=Curried(get_primitive_lambda(n_arguments, name[original])),
    )
    program = Program.parse(str(original))
    circuit = execute_program(program, arguments)

    # TODO: full_op_names should be fixed along with f_circuit
    full_op_names[
        name[original]
    ] = lambda tensor, *qubit, circuit_template=circuit: f_circuit(
        tensor, *qubit, circuit_template=circuit_template
    )
    
    simplified_program = Program.parse(str(simplified))
    simplified_circuit = execute_program(
        simplified_program, get_primitive_arguments(original.tp, [])
    )
    
    qk_circuit = get_qiskit_circuit(simplified_circuit).circuit
    

IndexError: tuple index out of range

In [ ]:
# Define primitive circuit operations as globals
for i, (original, simplified) in enumerate(simplification.items()):
    arguments = get_primitive_arguments(original.tp, [])
    n_qubit = len(arguments) - 1

    dc.program.Primitive.GLOBALS[name[original]] = dc.program.Primitive(
        name=name[original],
        ty=original.tp,
        value=dc.utilities.Curried(get_primitive_lambda(n_qubit, name[original])),
    )
    program = dc.program.Program.parse(str(original))

    # while 1:
    #     try:
    circuit = execute_program(program, arguments)
    #     break
    # except QuantumCircuitException:
    #     arguments =  randomize_arguments(arguments)
    #     print("Invalid circuit -> Randomizing arguments: ", arguments)

    dc.domains.quantum_circuits.primitives.full_op_names[
        name[original]
    ] = lambda tensor, *qubit, circuit_template=circuit: f_circuit(
        tensor, *qubit, circuit_template=circuit_template
    )

    simplified_program = dc.program.Program.parse(str(simplified))

    # TODO: problem with limited connectivity
    if experiment_arguments["limitedConnectivity"]:
        print(
            "Note that the simplified version can only have wrong arguments sometimes! Bug in the execution with constrained connectivity!"
        )
    simplified_circuit = execute_program(
        simplified_program, get_primitive_arguments(original.tp, [])
    )
    dc.domains.quantum_circuits.primitives.GLOBAL_LIMITED_CONNECTIVITY = (
        experiment_arguments["limitedConnectivity"]
    )

    qk_circuit = get_qiskit_circuit(simplified_circuit).circuit
    qk_circuit.name = name[original]
    dc.domains.quantum_circuits.primitives.qiskit_full_op_names[
        name[original]
    ] = get_qk_lambda(n_qubit, qk_circuit.to_gate())

    eta_long = dc.program.EtaLongVisitor().execute(simplified_program)
    print(f"{name[original]}: {eta_long}")  # EtaLong for a better program string
    # print(original)
    print("Simplified circuit")
    print(qk_circuit)
    qk_circuit.draw(output="mpl", filename=f"{folder_primitives}/{i}.pdf")
    print("Expanded circuit")
    print(get_qiskit_circuit(circuit).circuit)
    get_qiskit_circuit(circuit).circuit.draw(
        output="mpl", filename=f"{folder_primitives}/r{i}.pdf"
    )
    print("-------------------------------------------")

(cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (x ($0 (x ($0 (x ($0 (x (cz ($0 ($0 (cz ($0 ($0 (x (x $1 $2) $3) $4 $2) $5 $3) $2 $3) $6 $2) $7 $3) $2 $3) $2) $8 $3) $3) $9 $2) $2) $10 $3) $3) $11 $2) $2 $3) $12 $2) $2 $3) $13 $3) $2 $3) $14 $2) $2 $3) $15 $3) $2 $3) $16 $3) $2 $3) $17 $2) $2 $3)	


In [8]:
path

'akash/solved_RL_circuits/circ_list_TFIM_qubit2_J1_h0.001_decomposed1.pickle'

In [33]:
frontiers

[Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (sx (x (sx (cz (sx (rz (cz (rz (x (sx (rz (sx $0 $2) $5 $1) $1) $2) $4 $2) $1 $2) $3 $2) $2) $1 $2) $1) $2) $2))))))), logPrior=-33.92259025548687, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v0),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (sx (cz (x (x (sx (sx (rz (x (rz (x (rz (sx (cz (x (rz (sx (cz (rz (cz (rz (x (cz (sx (rz (x $0 $2) $9 $1) $2) $1 $2) $2) $8 $1) $1 $2) $7 $2) $1 $2) $2) $6 $1) $1) $1 $2) $1) $5 $2) $2) $4 $1) $1) $3 $2) $1) $2) $1) $2) $1 $2) $2))))))))))), logPrior=-76.26117218347213, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v1),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (x (sx (x (sx (x (rz (sx (rz (x (sx (sx (cz (x (cz (sx $0 $1) $1 $2) $1) $1 $2) $1) $2) $1) $4 $2) $2) $3 $1) $1) $1) $1) $1) $1)))))), logPrior=-38.92080302958456

In [163]:
instructions = get_instructions_from_qiskit(circuit)
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)
reconstructed = execute_program(program, arguments)

# Test

In [ ]:
# psi0 = np.array([1,0,0,0])
# rot = np.array(Operator(circ).data)
# psi1= np.dot(rot,psi0)

In [ ]:
# pipeline is
# CODE:str --> PROGRAM:Program --> INSTRUCTIONS:tuple --> CIRCUIT: QuantumCircuit

# inverse pipeline is
# CIRCUIT: QuantumCircuit --> INSTRUCTIONS: tuple --> CODE: str

In [ ]:
H = construct_hamiltonian(J, h, n_qubits)
for circ in b:
    psi0 = Statevector.from_int(0, 2**n_qubits)
    psi1 = psi0.evolve(circ)
    print(get_energy(psi1.data, H))

instructions = get_instructions_from_qiskit(circ)
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)

(-1.0000019905250335+0j)
(-1.000000389117297-5.551115123125783e-17j)
(-1.0000001942192795+0j)
(-1.0000000807637257+0j)
(-1.0000000793832347-5.551115123125783e-17j)
(-1.0000000000000002+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)


In [ ]:
# Test: get_qiskit_circuit(instructions) --> same circuit
instructions = get_instructions_from_qiskit(circ)
reconstructed_circuit = get_qiskit_circuit(instructions)
Operator(reconstructed_circuit.circuit).data - Operator(circ).data

In [ ]:
# Test: get code from instructions
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)
reconstructed = execute_program(program, arguments)
reconstructed == instructions

print(program)
print(reconstructed)
print(instructions)

In [16]:
primitives

[sx, x, rz, cz]

In [21]:
# Test: unitary from instructions
with QiskitTester(2) as QT:
    QT.circuit.rz(1, 1)
QT.circuit.draw()
instructions = get_instructions_from_qiskit(QT.circuit)
Operator(QT.circuit).data, circuit_to_mat(instructions)

(array([[0.87758256-0.47942554j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.87758256-0.47942554j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.87758256+0.47942554j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.87758256+0.47942554j]]),
 array([[0.87758256-0.47942554j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.87758256-0.47942554j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.87758256+0.47942554j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.87758256+0.47942554j]]))

In [ ]:
instructions = (2, (("sxdg", 0), ("x", 0), ("x", 1), ("rz", 1.43, 1), ("cnot", 0, 1)))
get_qiskit_circuit(instructions).circuit.draw()
# with QiskitTester(circuit=instructions) as QT:
#     QT.circuit.sxdg(QT.q(0))
# QT.circuit.draw()

┌───┐  ┌──────────┐┌───┐
q_0: ─┤ X ├──┤ Rz(1.43) ├┤ X ├
     ┌┴───┴─┐└──┬───┬───┘└─┬─┘
q_1: ┤ √Xdg ├───┤ X ├──────■──
     └──────┘   └───┘

In [ ]:
instructions = (
    1,
    (
        # ("sxdg", 0),
        # ("x", 0),
        # ("x", 1),
        ("rz", 1.43, 0),
        # ("cnot", 0,1)
    ),
)
get_qiskit_circuit(instructions).circuit.draw()
code, arguments = get_code_from_instructions(instructions)

In [ ]:
code = "(lambda (lambda(lambda (rz $0 $2 $1 ) )))"
code = "(cnot (no_op(2)) 1 0)"
code = "(cnot (I 2) 0 1)"
code = "(x (I 1) 0 )"
code = "(x (rz (I 1) 11 0) 0 )"

In [ ]:
program = Program.parse(code)
instructions = execute_program(program, ())
get_qiskit_circuit(instructions).circuit.draw()

Exception: Attempt to evaluate hole

In [ ]:
for circ in b:
    H = construct_hamiltonian(J, h, n_qubits)
    instructions = get_instructions_from_qiskit(circ)
    code, arguments = get_code_from_instructions(instructions)
    program = Program.parse(code)
    task = GroundStateTask(
        f"J_{J:2.2f}_h_{h:2.2f}_N_{n_qubits}",
        hamiltonian=H,
        arguments=arguments,
        request=program.infer(),
    )
    energy = task.logLikelihood(program)

    psi0 = Statevector.from_int(0, 4)
    psi1 = psi0.evolve(circ)
    print(get_energy(psi1, H), energy)

(-1.0000019905250335+0j) 1.0000019905250335
(-1.000000389117297-5.551115123125783e-17j) 1.0000003891172966
(-1.0000001942192795+0j) 1.0000001942192795
(-1.0000000807637257+0j) 1.0000000807637257
(-1.0000000793832347-5.551115123125783e-17j) 1.000000079383235
(-1.0000000000000002+0j) 1.000000000000001
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007


In [ ]:
instructions

(2,
 (('x', 0),
  ('rz', 12.852044105529785, 1),
  ('sx', 0),
  ('rz', 4.629762649536133, 0),
  ('cz', 1, 0),
  ('rz', 5.277709484100342, 1),
  ('cz', 1, 0),
  ('sx', 0),
  ('rz', 0.030434206128120422, 1),
  ('x', 1),
  ('x', 0),
  ('rz', -1.6001750230789185, 1),
  ('x', 1),
  ('cz', 1, 0),
  ('x', 1),
  ('x', 0),
  ('cz', 1, 0),
  ('rz', -0.9125978946685791, 1),
  ('cz', 1, 0),
  ('sx', 0),
  ('x', 1),
  ('rz', -1.3367031812667847, 1),
  ('rz', -1.1753524541854858, 0),
  ('x', 1),
  ('cz', 1, 0),
  ('sx', 1),
  ('x', 0),
  ('cz', 1, 0),
  ('sx', 1)))